In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline  

## Dataset
Pandas provide a .read_csv() function. the paramter engine='python' was needed here.
We can see in the output that there are two columns "text" and "spam" 

In [7]:
data = pd.read_csv('/home/shreyansh/Downloads/emails.csv', encoding='latin-1', engine='python')
data.head(n=10)
print(data)

                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
5     Subject: great nnews  hello , welcome to medzo...     1
6     Subject: here ' s a hot play in motion  homela...     1
7     Subject: save your money buy getting this thin...     1
8     Subject: undeliverable : home based business f...     1
9     Subject: save your money buy getting this thin...     1
10    Subject: las vegas high rise boom  las vegas i...     1
11    Subject: save your money buy getting this thin...     1
12    Subject: brighten those teeth  get your  teeth...     1
13    Subject: wall street phenomenon reaps rewards ...     1
14    Subject: fpa notice : ebay misrepresentation o...     1
15    Su

In [22]:
f = TfidfVectorizer(stop_words = 'english')
X = f.fit_transform(data["text"])
print(np.shape(X))
print(X)

(5728, 36996)
  (0, 32145)	0.01781272601181811
  (0, 23219)	0.12256415839423043
  (0, 18705)	0.1432830450797234
  (0, 9986)	0.07425233664021534
  (0, 17562)	0.09302659600058759
  (0, 21006)	0.11259587569184905
  (0, 27817)	0.06817457798935368
  (0, 16546)	0.07848771947213565
  (0, 27941)	0.11391594069511475
  (0, 9223)	0.16756208708238118
  (0, 21520)	0.11577281792556753
  (0, 32408)	0.12371376538846193
  (0, 18103)	0.04504514237999522
  (0, 18751)	0.13491099920740895
  (0, 15964)	0.1033087647302782
  (0, 7986)	0.11461456851250225
  (0, 20818)	0.265157481584021
  (0, 32126)	0.12371376538846193
  (0, 31776)	0.12256415839423043
  (0, 24679)	0.09199667182622404
  (0, 35805)	0.13645440111292698
  (0, 21296)	0.09831341956555792
  (0, 32839)	0.09815099793568317
  (0, 12539)	0.09281579204902306
  (0, 26937)	0.20113998716066434
  :	:
  (5727, 24659)	0.10273386204392845
  (5727, 21490)	0.04981665317903681
  (5727, 5683)	0.557612387822799
  (5727, 30755)	0.052872641241471056
  (5727, 2807)	0.176

In [15]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, data['spam'], test_size=0.33, random_state=42)
print([np.shape(X_train), np.shape(X_test)])

[(3837, 36996), (1891, 36996)]


In [16]:
list_C = np.arange(500, 2000, 100) #100000
score_train = np.zeros(len(list_C))
score_test = np.zeros(len(list_C))
recall_test = np.zeros(len(list_C))
precision_test= np.zeros(len(list_C))
count = 0
for C in list_C:
    svc = svm.SVC(C=C)
    svc.fit(X_train, y_train)
    score_train[count] = svc.score(X_train, y_train)
    score_test[count]= svc.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, svc.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, svc.predict(X_test))
    count = count + 1 

In [17]:
matrix = np.matrix(np.c_[list_C, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['C', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

,C,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,500.0,0.771957,0.739820,0.002028,1.0
1,600.0,0.787594,0.754627,0.058824,1.0
2,700.0,0.816784,0.790058,0.194726,1.0
3,800.0,0.820172,0.792702,0.204868,1.0
4,900.0,0.829033,0.801163,0.237323,1.0
5,1000.0,0.840240,0.809096,0.267748,1.0
6,1100.0,0.849622,0.818614,0.304260,1.0
7,1200.0,0.864217,0.832364,0.356998,1.0
8,1300.0,0.876466,0.843469,0.399594,1.0
9,1400.0,0.888976,0.852988,0.436105,1.0


In [18]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

C                 500.000000
Train Accuracy      0.771957
Test Accuracy       0.739820
Test Recall         0.002028
Test Precision      1.000000
Name: 0, dtype: float64

In [19]:
models[models['Test Precision']==1].head(n=5)

,C,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,500.0,0.771957,0.739820,0.002028,1.0
1,600.0,0.787594,0.754627,0.058824,1.0
2,700.0,0.816784,0.790058,0.194726,1.0
3,800.0,0.820172,0.792702,0.204868,1.0
4,900.0,0.829033,0.801163,0.237323,1.0


In [20]:
best_index = models[models['Test Precision']==1]['Test Accuracy'].idxmax()
svc = svm.SVC(C=list_C[best_index])
svc.fit(X_train, y_train)
models.iloc[best_index, :]

C                 1700.000000
Train Accuracy       0.925463
Test Accuracy        0.879958
Test Recall          0.539554
Test Precision       1.000000
Name: 12, dtype: float64

In [21]:
m_confusion_test = metrics.confusion_matrix(y_test, svc.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,1398,0
Actual 1,227,266
